<a href="https://colab.research.google.com/github/Youka419/manabidx_kadai03_do/blob/main/%E6%BC%94%E7%BF%9203%E3%80%80%E5%9F%BA%E6%9C%AC%E3%82%B3%E3%83%BC%E3%83%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

履歴を保存するPythonコード

In [3]:
!pip install japanize_matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 52.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=0321572ae109248806ac919193366b9f73dbec4f34e26e37e227f06f107d762c
  Stored in directory: /root/.cache/pip/wheels/c1/f7/9b/418f19a7b9340fc16e071e89efc379aca68d40238b258df53d
Successfully built japanize_matplotlib


In [1]:
# 必要なライブラリをインポート
import os
from datetime import datetime
import shutil

# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# Colab Notebooks フォルダのパス
colab_notebooks_path = '/content/drive/MyDrive/Colab Notebooks'

# 日付ごとのフォルダを作成
# 例: 2025-08-24_10-30-00
history_folder = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
history_path = os.path.join(colab_notebooks_path, 'history', history_folder)

# フォルダが存在しない場合は作成
os.makedirs(history_path, exist_ok=True)

# 現在のノートブックのパスを取得
# Jupyter NotebookのAPIを利用して、実行中のノートブックのパスを取得します
# このコードはColab上でのみ動作します
try:
    from google.colab import files
    notebook_path = os.path.join(os.getcwd(), 'theme3_sample_code_v3.ipynb')
    shutil.copy(notebook_path, history_path)
    print(f"現在のノートブックを '{history_path}' にコピーしました。")
except Exception as e:
    print(f"ノートブックのコピー中にエラーが発生しました: {e}")

Mounted at /content/drive
ノートブックのコピー中にエラーが発生しました: [Errno 2] No such file or directory: '/content/theme3_sample_code_v3.ipynb'


1. 必要なライブラリの読み込み

In [5]:
# データ分析を効率的に行うためのライブラリ
import pandas as pd
import numpy as np

# データをグラフとして可視化するためのライブラリ
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns

# 機械学習のためのライブラリ
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

2. データ読み込みと前処理

In [13]:
# データをpandasのDataFrameとして読み込む
# Google Driveのマイドライブにある「MDXQ」フォルダのパスを指定
drive_path = '/content/drive/MyDrive/MDXQ'
train_file_path = drive_path + '/train.csv'
test_file_path = drive_path + '/test.csv'

train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)

# 日付に関する項目をdatetime型に変換
train_df['基準年月'] = pd.to_datetime(train_df['基準年月'])
# test_dfには日付に関する明確な列がないため、test_dfの日付変換はコメントアウトします。
test_df['基準年月'] = pd.to_datetime(test_df['基準年月']) # test_dfにも基準年月列があるため追加

# 年と月を別の列に抽出
train_df['基準年'] = train_df['基準年月'].dt.year
train_df['基準月'] = train_df['基準年月'].dt.month
test_df['基準年'] = test_df['基準年月'].dt.year # test_dfにも年を抽出
test_df['基準月'] = test_df['基準年月'].dt.month # test_dfにも月を抽出

# 欠損値の補完や、モデルが扱える形式への変換など、必要に応じて前処理をここに記述
# 例: カテゴリ変数を数値に変換する
# train_df['商品カテゴリ'] = train_df['商品カテゴリ'].astype('category').cat.codes

display(train_df.head())
display(test_df.head())

,ID,顧客ID,住所コード,顧客氏名,基準年月,取得価額,時価価額,評価損益,投資方針,train_term_1,...,train_term_4,train_term_5,train_term_6,翌月_購入,翌月_売却,翌々月_購入,翌々月_売却,y,基準年,基準月
0,10001,1,A-1,平野勇,2015-05-01,235.0,441.15,206.15,2,1,...,1,1,1,0,0,0,0,0,2015,5
1,10002,2,A-1,栗原直人,2015-05-01,0.0,0.00,0.00,2,1,...,1,1,1,0,0,0,0,0,2015,5
2,10003,3,A-1,川口結衣,2015-05-01,535.0,540.24,5.24,2,1,...,1,1,1,0,0,0,0,0,2015,5
3,10004,4,A-1,森真希,2015-05-01,713.0,899.86,186.85,3,1,...,1,1,1,0,0,0,0,0,2015,5
4,10005,5,A-1,後藤歩,2015-05-01,48.0,48.14,0.14,1,1,...,1,1,1,0,0,0,0,0,2015,5


,Unnamed: 0,ID,顧客ID,住所コード,顧客氏名,基準年月,取得価額,時価価額,評価損益,投資方針,test_term_1,test_term_2,test_term_3,test_term_4,test_term_5,test_term_6,基準年,基準月
0,82000,830001,1,A-1,平野勇,2022-03-01,0.0,0.00,0.00,2,1,0,0,0,0,0,2022,3
1,82001,830002,2,A-1,栗原直人,2022-03-01,0.0,0.00,0.00,2,1,0,0,0,0,0,2022,3
2,82002,830003,3,A-1,川口結衣,2022-03-01,768.0,795.58,27.58,2,1,0,0,0,0,0,2022,3
3,82003,830004,4,A-1,森真希,2022-03-01,1259.0,1593.91,334.91,3,1,0,0,0,0,0,2022,3
4,82004,830005,5,A-1,後藤歩,2022-03-01,103.0,102.06,-0.94,1,1,0,0,0,0,0,2022,3


3. 特徴量とターゲットの作成

In [11]:
# 特徴量として使用する列名をリストで定義
features_for_model = [
    # ここに利用したい特徴量の列名を追加していく
    # 例: '時価価額', '評価損益', '投資経験（株式）'
    '時価価額', '評価損益', '投資方針', '基準年', '基準月' # 例としていくつかの列を追加
]

# 学習データから特徴量 (X) を作成
X = train_df[features_for_model]

# 学習データからターゲット (y) を作成
# ターゲットは取引があったかどうかを示す目的変数
y = train_df['y']

4. モデルの学習と予測

In [14]:
# LightGBMモデルのインスタンス化と学習
model = lgb.LGBMClassifier(random_state=42)
model.fit(X, y)

# テストデータで予測
X_test = test_df[features_for_model]
y_pred = model.predict_proba(X_test)[:, 1]

[LightGBM] [Info] Number of positive: 17899, number of negative: 72101
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002211 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 536
[LightGBM] [Info] Number of data points in the train set: 90000, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.198878 -> initscore=-1.393323
[LightGBM] [Info] Start training from score -1.393323


5. 予測結果の出力

In [15]:
# 提出用のデータフレームを作成し、予測結果を格納
submission_df = pd.DataFrame({'ID': test_df['ID'], 'predict': y_pred})

# CSVファイルとして保存
submission_df.to_csv('submission.csv', index=False)

6.出力データの保存先変更

In [16]:
import os
from datetime import datetime

# 実行日時のフォルダ名を生成
# 例: 2025-08-24
output_folder = datetime.now().strftime('%Y-%m-%d')
output_path = os.path.join('/content/drive/MyDrive/Colab Notebooks', output_folder)

# フォルダが存在しない場合は作成
os.makedirs(output_path, exist_ok=True)

# 提出用データフレームを作成
submission_df = pd.DataFrame({'ID': test_df['ID'], 'predict': y_pred})

# 提出用ファイル名を生成
submission_file_name = f'my_submission_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv'
submission_file_path = os.path.join(output_path, submission_file_name)

# ローカルに保存（指定したフォルダ内）
submission_df.to_csv(submission_file_path, index=False, header=False)

print(f"\n'{submission_file_path}' にファイルを保存しました。")


'/content/drive/MyDrive/Colab Notebooks/2025-08-23/my_submission_20250823_231040.csv' にファイルを保存しました。


履歴を保存するPythonコード

In [17]:
# 必要なライブラリをインポート
import os
from datetime import datetime
import shutil

# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# Colab Notebooks フォルダのパス
colab_notebooks_path = '/content/drive/MyDrive/Colab Notebooks'

# 日付ごとのフォルダを作成
# 例: 2025-08-24_10-30-00
history_folder = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
history_path = os.path.join(colab_notebooks_path, 'history', history_folder)

# フォルダが存在しない場合は作成
os.makedirs(history_path, exist_ok=True)

# 現在のノートブックのパスを取得
# Jupyter NotebookのAPIを利用して、実行中のノートブックのパスを取得します
# このコードはColab上でのみ動作します
try:
    from google.colab import files
    notebook_path = os.path.join(os.getcwd(), 'theme3_sample_code_v3.ipynb')
    shutil.copy(notebook_path, history_path)
    print(f"現在のノートブックを '{history_path}' にコピーしました。")
except Exception as e:
    print(f"ノートブックのコピー中にエラーが発生しました: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ノートブックのコピー中にエラーが発生しました: [Errno 2] No such file or directory: '/content/theme3_sample_code_v3.ipynb'


In [18]:
# データ分析を効率的に行うためのライブラリ
import pandas as pd
import numpy as np

# データをグラフとして可視化するためのライブラリ
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns

# 機械学習のためのライブラリ
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb